In [1]:
import pandas as pd
import numpy as np

In [2]:
Schools_input=pd.read_csv("HWK 04 Schools.csv")
Students_input=pd.read_csv("HWK 04 Students.csv")     

In [3]:
#BASIC ANALYSIS ON SCHOOLS INPUT

#Schools_input.head()
#Schools_input.dtypes
#Schools_input["type"].unique() = array(['District', 'Charter'], dtype=object)
#Schools_input.describe()

In [4]:
#BASIC ANALYSIS ON STUDENTS INPUT

#Students_input.head()
#Students_input.dtypes
#Students_input["gender"].unique() = array(['M', 'F'], dtype=object)
#Students_input["grade"].unique() = array(['9th', '12th', '11th', '10th'], dtype=object)
#Students_input["school_name"].unique()
#Students_input.describe()

In [5]:
#MERGE SCHOOLS_INPUT AND STUDENTS_INPUT IN ONE TABLE
Schools_Students_df=Schools_input.merge(Students_input,on="school_name")

In [6]:
#1.0 DISTRICT SUMMARY

#Total Schools
Schools=Schools_input["school_name"].unique()
Total_Schools=len(Schools)

#Total Students
Total_Students=len(Students_input["student_name"])

#Total Budget
Total_Budget = Schools_input["budget"].sum()

#Average Math Score
AVG_Math=Students_input["math_score"].mean()

#Average Reading Score
AVG_Read=Students_input["reading_score"].mean()

#%Passing Math (Above or equal to 70)
Students_Math_Pass=Students_input.loc[Students_input["math_score"]>69,:]
PC_Math_Pass=len(Students_Math_Pass["student_name"])/Total_Students
PC_Math_Pass

#%Passing Reading (ABove or equal to 70)
Students_Read_Pass=Students_input.loc[Students_input["reading_score"]>69,:]
PC_Read_Pass=len(Students_Read_Pass["student_name"])/Total_Students
PC_Read_Pass

#Overall Passing Rate
Passing_Rate=(PC_Math_Pass+PC_Read_Pass)/2

#Labels for District Snapshot Table
Labels=["Total Schools","Total Students","Total Budget","Math Average","Reading Average","% Passing Math","% Passing Reading",
       "Overall Passing Rate"]

#Values for the District Snapshot Table
Values=["{:,}".format(Total_Schools),
        "{:,}".format(Total_Students),
        "$"+"{:,}".format(round(Total_Budget,2)),
        round(AVG_Math,2),
        round(AVG_Read,2),
        "{0:.2%}".format(PC_Math_Pass),
        "{0:.2%}".format(PC_Read_Pass),
        "{0:.2%}".format(Passing_Rate)]

District_Snapshot=pd.DataFrame(np.array([Values]),columns=Labels)
District_Snapshot

,Total Schools,Total Students,Total Budget,Math Average,Reading Average,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.39%


In [8]:
#2.0 SCHOOL SUMMARY

Schools_Summary=Schools_input

#2.01 Per Student Budget added
Schools_Summary["Budget/Student"]=Schools_input["budget"]/Schools_input["size"] 

#2.02 Averages of Readig and Math Scores per school
AVGs_School=Schools_Students_df.groupby(["school_name"]).mean()
AVGs_School=AVGs_School.rename(columns={"reading_score":"Average Reading Score","math_score":"Average Math Score"})
AVGs_School=AVGs_School.loc[:,["Average Reading Score","Average Math Score"]]

#2.02.1 Adds averages to Summary Table
Schools_Summary=Schools_input.merge(AVGs_School,on="school_name")

#2.03 Calculates total students passing Math
School_Math_Pass=Students_Math_Pass.groupby(["school_name"],as_index=False).count()
School_Math_Pass=School_Math_Pass.loc[:,["school_name","student_name"]]
School_Math_Pass=School_Math_Pass.rename(columns={"student_name":"Students Pass Math"})

#2.03.1 Adds Column of Students passing Math to Summary Table
Schools_Summary=Schools_Summary.merge(School_Math_Pass,on="school_name")

#2.04 Calculates total students passing Reading
School_Read_Pass=Students_Read_Pass.groupby(["school_name"],as_index=False).count()
School_Read_Pass=School_Read_Pass.loc[:,["school_name","student_name"]]
School_Read_Pass=School_Read_Pass.rename(columns={"student_name":"Students Pass Reading"})

#2.04.1 Adds Column of Students passing Reading to Summary Table
Schools_Summary=Schools_Summary.merge(School_Read_Pass,on="school_name")

#2.05 Calculates % of Students that passed Math
Schools_Summary["% Passing Math"]=Schools_Summary["Students Pass Math"]/Schools_Summary["size"]

#2.06 Calculates % of Students that passed Reading
Schools_Summary["% Passing Reading"]=Schools_Summary["Students Pass Reading"]/Schools_Summary["size"]

#2.07 Calculates the Overall Passing Range as the Average of "% Passing Math" and "% Passing Reading"
Schools_Summary["Overall Passing Rate"]=(Schools_Summary["% Passing Reading"]+Schools_Summary["% Passing Math"])/2

#2.08 Deletes unnecesary Columns ("School ID", "Students Pass Math" and "Students Pass Reading")
Schools_Summary=Schools_Summary.drop(["Students Pass Math","Students Pass Reading","School ID"],axis=1)

#2.09 Sets the Scholl Name as Index
Schools_Summary=Schools_Summary.set_index("school_name")

#2.10 Gives Format to the Table

#2.10.1 Budget, Per Student Budget as Currency ("${:,.2f}")
Schools_Summary["budget"]=Schools_Summary["budget"].map("${:,.2f}".format)
Schools_Summary["Budget/Student"]=Schools_Summary["Budget/Student"].map("${:,.2f}".format)

#2.10.2 % Passing Math, % Passing Reading and Overall Passing Rate as Percentage ("{0:.3%}")
Schools_Summary["% Passing Math"]=Schools_Summary["% Passing Math"].map("{0:.3%}".format)
Schools_Summary["% Passing Reading"]=Schools_Summary["% Passing Reading"].map("{0:.3%}".format)
Schools_Summary["Overall Passing Rate"]=Schools_Summary["Overall Passing Rate"].map("{0:.3%}".format)

Schools_Summary

,type,size,budget,Budget/Student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.684%,81.316%,73.500%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988%,80.739%,73.364%
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.725724,83.359455,93.867%,95.855%,94.861%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,66.753%,80.863%,73.808%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392%,97.139%,95.266%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,93.868%,96.540%,95.204%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133%,97.040%,95.587%
Bailey High School,District,4976,"$3,124,928.00",$628.00,81.033963,77.048432,66.680%,81.933%,74.307%
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.506%,96.253%,94.379%


In [9]:
#3.0 TOP PERFORMERS

#3.01 Sorts from higest Overall Passing Rate to Lowest Overall Passing Rage and filters first 5 Performers
Top_Performers=(Schools_Summary.sort_values(by ="Overall Passing Rate",ascending=False)).head(5)
Top_Performers

,type,size,budget,Budget/Student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133%,97.040%,95.587%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,93.272%,97.309%,95.291%
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.595%,95.946%,95.270%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392%,97.139%,95.266%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,93.868%,96.540%,95.204%


In [11]:
#4.0 BOTTOM PERFORMERS

#4.01 Sorts from lowest Overall Passing Rate to highest Overall Passing Rage and filters first 5 Performers
Bottom_Performers=(Schools_Summary.sort_values(by ="Overall Passing Rate",ascending=True)).head(5)
Bottom_Performers

,type,size,budget,Budget/Student,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,66.367%,80.220%,73.293%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988%,80.739%,73.364%
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.684%,81.316%,73.500%
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.058%,81.222%,73.640%
Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,68.310%,79.299%,73.804%


In [12]:
# 5.0 MATH SCORES BY GRADE

#5.01 Generate a shorter DataFrame with only School Name, Grade and Math Score
Math_info=Schools_Students_df.loc[:,["school_name","grade","math_score"]]

#5.02 Calculate the mean of the math_score for each School
AVG_Math =Math_info.groupby(["school_name"]).mean()

#5.03 Defines all the Grades in the database and lists them into the Variable "Grades"
Grades=Students_input["grade"].unique()

#5.04 Itinerate in each grade found in Grades, determine the average of math_score and add a column with the information
#in the final table AVG_Math
for i in Grades:
    Math_Grade=Math_info.loc[Math_info["grade"]==i,:]
    AVG_Math_Grade=Math_Grade.groupby(["school_name"]).mean()
    AVG_Math[i]=AVG_Math_Grade

#5.05 Drop math_score since it is not needed
AVG_Math=AVG_Math.drop(["math_score"],axis=1)

#5.05 Arrange columns from lowest grade to highest
AVG_Math=AVG_Math[["9th","10th","11th","12th"]]
AVG_Math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [13]:
#6.0 READING SCORES BY GRADE

#6.01 Generate a shorter DataFrame with only School Name, Grade and Reading Score
Read_info=Schools_Students_df.loc[:,["school_name","grade","reading_score"]]

#6.02 Calculate the mean of the reading_score for each School
AVG_Read =Read_info.groupby(["school_name"]).mean()

#6.03 Itinerate in each grade found in Grades, determine the average of reading_score and add a column with the information
#in the final table AVG_Read
for i in Grades:
    Read_Grade=Read_info.loc[Read_info["grade"]==i,:]
    AVG_Read_Grade=Read_Grade.groupby(["school_name"]).mean()
    AVG_Read[i]=AVG_Read_Grade

#6.04 Drop reading_score since it is not needed
AVG_Read=AVG_Read.drop(["reading_score"],axis=1)

#6.05 Arrange columns from lowest grade to highest
AVG_Read=AVG_Read[["9th","10th","11th","12th"]]

AVG_Read

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [44]:
#7.0 SCORES BY SCHOOL SPENDING

#7.01 Defines ranges for bins
Budget_bins=[0,592,628,641,655]

#7.01.1 Defines descriptive labels for the bins defined
Budget_Range=["<$"+str(Budget_bins[1]),
                "$"+str(Budget_bins[1])+"-$"+str(Budget_bins[2]),
                "$"+str(Budget_bins[2])+" -$"+str(Budget_bins[3]),
                "$"+str(Budget_bins[3])+" - $"+str(Budget_bins[4])]

#7.02 Categorize each school according to its Per Student Budget Range
Scores_by_School_Spending=Schools_input
Scores_by_School_Spending["Budget/Student Range"]=pd.cut(Scores_by_School_Spending["Budget/Student"],Budget_bins)

#7.02.01 If needed, counts how many schools were categorized in each range
#print(Scores_by_School_Spending.groupby("Budget/Student Range").count())

#7.02.02 Groups by the "Budget/Student Range" as an addition to obtain the total students each range has
Scores_by_School_Spending=Scores_by_School_Spending.groupby("Budget/Student Range").sum()

#7.02.03 Drops "School ID", "Per Student Budget" colums since they are not needed
Scores_by_School_Spending=Scores_by_School_Spending.drop(["School ID","Budget/Student"],axis=1)

#7.03 Obtain scores for each Per Student Budget Range

#7.03.01 Generate a shorter DataFrame with only information needed
Budget_per_Student=Schools_Students_df.loc[:,["size","budget","reading_score","math_score"]]

#7.03.02 Calculate the Budget per Student
Budget_per_Student["Budget/Student"]=Budget_per_Student["budget"]/Budget_per_Student["size"]

#7.03.03 Categorize each score according tho the Schools Budget per Student Range
Budget_per_Student["Budget/Student Range"]=pd.cut(Budget_per_Student["Budget/Student"],Budget_bins)

#7.04 Determine average Reading and Math scores in each Per Student Budget Range
AVG_Scores_per_Budget_Range=Budget_per_Student.groupby(["Budget/Student Range"],as_index=False).mean()
AVG_Scores_per_Budget_Range=AVG_Scores_per_Budget_Range.loc[:,["Budget/Student Range","reading_score","math_score"]]

#7.04.01 Merge with the main table (Scores_by_School_Spending)
Scores_by_School_Spending=Scores_by_School_Spending.merge(AVG_Scores_per_Budget_Range,on="Budget/Student Range")

#7.05 Calculate the total of students that passed Math (70 or higher)
Budget_Range_Pass_Math=Budget_per_Student.loc[Budget_per_Student["math_score"]>69,:]
Budget_Range_Pass_Math=Budget_Range_Pass_Math.groupby(["Budget/Student Range"],as_index=False).count()
Budget_Range_Pass_Math=Budget_Range_Pass_Math.loc[:,["Budget/Student Range","size"]]
Budget_Range_Pass_Math=Budget_Range_Pass_Math.rename(columns={"size":"Students Passing Math"})

#7.05.01 Merge with the main table (Scores_by_School_Spending)
Scores_by_School_Spending=Scores_by_School_Spending.merge(Budget_Range_Pass_Math,on="Budget/Student Range")
Scores_by_School_Spending

#7.06 Calculate the total of students that passed Math (70 or higher)
Budget_Range_Pass_Read=Budget_per_Student.loc[Budget_per_Student["reading_score"]>69,:]
Budget_Range_Pass_Read=Budget_Range_Pass_Read.groupby(["Budget/Student Range"],as_index=False).count()
Budget_Range_Pass_Read=Budget_Range_Pass_Read.loc[:,["Budget/Student Range","size"]]
Budget_Range_Pass_Read=Budget_Range_Pass_Read.rename(columns={"size":"Students Passing Reading"})

#7.06.01 Merge with the main table (Scores_by_School_Spending)
Scores_by_School_Spending=Scores_by_School_Spending.merge(Budget_Range_Pass_Read,on="Budget/Student Range")
Scores_by_School_Spending

#7.07 Calculates % of Students that passed Math
Scores_by_School_Spending["% Passing Math"]=Scores_by_School_Spending["Students Passing Math"]/Scores_by_School_Spending["size"]

#7.08 Calculate % of Students that passed Reading
Scores_by_School_Spending["% Passing Reading"]=Scores_by_School_Spending["Students Passing Reading"]/Scores_by_School_Spending["size"]

#7.09 Calculate the Overall Passing Rate
Scores_by_School_Spending["Overall Passing Rate"]=(Scores_by_School_Spending["% Passing Math"]+Scores_by_School_Spending["% Passing Reading"])/2

#7.10 Keeps only necessary columns
Scores_by_School_Spending=Scores_by_School_Spending.loc[:,["Budget/Student Range","reading_score","math_score",
                                                       "% Passing Math","% Passing Reading","Overall Passing Rate"]]
#7.11 Gives Format to the Table

#7.11.1 % Passing Math, % Passing Reading and Overall Passing Rate as Percentage ("{0:.3%}")
Scores_by_School_Spending["% Passing Math"]=Scores_by_School_Spending["% Passing Math"].map("{0:.3%}".format)
Scores_by_School_Spending["% Passing Reading"]=Scores_by_School_Spending["% Passing Reading"].map("{0:.3%}".format)
Scores_by_School_Spending["Overall Passing Rate"]=Scores_by_School_Spending["Overall Passing Rate"].map("{0:.3%}".format)

#7.11.2 Renames Columns
Scores_by_School_Spending=Scores_by_School_Spending.rename(columns={"reading_score":"Avg Reading Score",
                                                                    "math_score":"Avg Math Score"})

#7.11.3 Adds Budget_Rage Labels as index
Scores_by_School_Spending["Budget Range"]=Budget_Range
Scores_by_School_Spending=Scores_by_School_Spending.set_index("Budget Range")
Scores_by_School_Spending=Scores_by_School_Spending.drop(["Budget/Student Range"],axis=1)

#Budget_per_Student.head()
Scores_by_School_Spending
#AVG_Scores_per_Budget_Range
#Budget_Range_Pass_Math
#Budget_Range_Pass_Read

,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
Budget Range,,,,,
<$592,83.964039,83.363065,93.703%,96.687%,95.195%
$592-$628,82.312643,79.982873,79.110%,88.513%,83.811%
$628 -$641,81.478038,78.050332,71.362%,83.654%,77.508%
$641 - $655,80.958411,77.058995,66.609%,80.780%,73.695%


In [60]:
#8.0 SCORES BY SCHOOL SIZE

#8.01 Defines ranges for bins
Size_bins=[0,1500,3000,5000]

#8.01.1 Defines descriptive labels for the bins defined
Size_Range=["Small","Medium","Large"]

#8.02 Categorize each school according to its Size
Scores_by_School_Size=Schools_input
Scores_by_School_Size["School Size"]=pd.cut(Scores_by_School_Size["size"],Size_bins)

#8.02.01 If needed, counts how many schools were categorized in each range
#print(Scores_by_School_Size.groupby("School Size").count())

#8.02.02 Groups by the "School Size" as an addition to obtain the total students each range has
Scores_by_School_Size=Scores_by_School_Size.groupby("School Size").sum()

#8.02.03 Drops "School ID", "Budget/Student", "budget" colums since they are not needed
Scores_by_School_Size=Scores_by_School_Size.drop(["budget","Budget/Student","School ID"],axis=1)

#8.03 Obtain scores for each School Size

#8.03.01 Generate a shorter DataFrame with only information needed
Per_School_Size=Schools_Students_df.loc[:,["size","reading_score","math_score"]]

#8.03.02 Categorize each score according tho the Schools Size
Per_School_Size["School Size"]=pd.cut(Per_School_Size["size"],Size_bins)

#8.04 Determine average Reading and Math scores in each School Size and keeps the needed columns
AVG_per_Size=Per_School_Size.groupby(["School Size"],as_index=False).mean()
AVG_per_Size=AVG_per_Size.loc[:,["School Size","reading_score","math_score"]]

#8.04.01 Merge with the main table (Scores_by_School_Size)
Scores_by_School_Size=Scores_by_School_Size.merge(AVG_per_Size,on="School Size")

#8.05 Calculate the total of students that passed Math (70 or higher)
PassMath_per_Size=Per_School_Size.loc[Per_School_Size["math_score"]>69,:]
PassMath_per_Size=PassMath_per_Size.groupby(["School Size"],as_index=False).count()
PassMath_per_Size=PassMath_per_Size.loc[:,["School Size","size"]]
PassMath_per_Size=PassMath_per_Size.rename(columns={"size":"Students Passing Math"})

#8.05.01 Merge with the main table (Scores_by_School_Size)
Scores_by_School_Size=Scores_by_School_Size.merge(PassMath_per_Size,on="School Size")

#8.06 Calculate the total of students that passed Reading (70 or higher)
PassRead_per_Size=Per_School_Size.loc[Per_School_Size["reading_score"]>69,:]
PassRead_per_Size=PassRead_per_Size.groupby(["School Size"],as_index=False).count()
PassRead_per_Size=PassRead_per_Size.loc[:,["School Size","size"]]
PassRead_per_Size=PassRead_per_Size.rename(columns={"size":"Students Passing Reading"})

#8.06.01 Merge with the main table (Scores_by_School_Size)
Scores_by_School_Size=Scores_by_School_Size.merge(PassRead_per_Size,on="School Size")

#8.07 Calculates % of Students that passed Math
Scores_by_School_Size["% Passing Math"]=Scores_by_School_Size["Students Passing Math"]/Scores_by_School_Size["size"]

#8.08 Calculates % of Students that passed Reading
Scores_by_School_Size["% Passing Reading"]=Scores_by_School_Size["Students Passing Reading"]/Scores_by_School_Size["size"]

#8.09 Calculate the Overall Passing Rate
Scores_by_School_Size["Overall Passing Rate"]=(Scores_by_School_Size["% Passing Math"]+Scores_by_School_Size["% Passing Reading"])/2

#8.10 Keeps only necessary columns
Scores_by_School_Size=Scores_by_School_Size.loc[:,["School Size","reading_score","math_score",
                                                       "% Passing Math","% Passing Reading","Overall Passing Rate"]]

#8.11 Gives Format to the Table

#8.11.1 % Passing Math, % Passing Reading and Overall Passing Rate as Percentage ("{0:.3%}")
Scores_by_School_Size["% Passing Math"]=Scores_by_School_Size["% Passing Math"].map("{0:.3%}".format)
Scores_by_School_Size["% Passing Reading"]=Scores_by_School_Size["% Passing Reading"].map("{0:.3%}".format)
Scores_by_School_Size["Overall Passing Rate"]=Scores_by_School_Size["Overall Passing Rate"].map("{0:.3%}".format)

#8.11.2 Renames Columns
Scores_by_School_Size=Scores_by_School_Size.rename(columns={"reading_score":"Avg Reading Score","math_score":"Avg Math Score"})

#8.11.3 Adds Size Labels as index
Scores_by_School_Size["Size"]=Size_Range
Scores_by_School_Size=Scores_by_School_Size.set_index("Size")
Scores_by_School_Size=Scores_by_School_Size.drop(["School Size"],axis=1)

#PassRead_per_Size
#PassMath_per_Size
#AVG_per_Size
#Per_School_Size.head()
Scores_by_School_Size

,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size,,,,,
Small,83.893245,83.583479,93.665%,96.605%,95.135%
Medium,82.529094,80.213577,80.721%,88.898%,84.809%
Large,80.928365,77.070764,66.469%,81.106%,73.787%


In [74]:
#9.0 SCORES BY SCHOOL TYPE

#9.01 Groups by the "type" as an addition to obtain the total students each type has and keeps the needed columns
Scores_by_Type=Schools_input
Scores_by_Type=Scores_by_Type.groupby("type").sum()
Scores_by_Type=Scores_by_Type.drop(["budget","Budget/Student","School ID"],axis=1)

#9.02 Determines average Reading and Math scores in each School Size and keeps the needed columns
AVG_by_Type=Schools_Students_df.groupby("type").mean()
AVG_by_Type=AVG_by_Type.loc[:,["reading_score","math_score"]]

#9.02.01 Merges with the main table (Scores_by_Type)
Scores_by_Type=Scores_by_Type.merge(AVG_by_Type,on="type")

#9.03 Calculate the total of students that passed Math (70 or higher)
PassMath_by_Type=Schools_Students_df.loc[Per_School_Size["math_score"]>69,:]
PassMath_by_Type=PassMath_by_Type.groupby(["type"],as_index=False).count()
PassMath_by_Type=PassMath_by_Type.loc[:,["type","size"]]
PassMath_by_Type=PassMath_by_Type.rename(columns={"size":"Students Passing Math"})

#9.03.01 Merges with the main table (Scores_by_Type)
Scores_by_Type=Scores_by_Type.merge(PassMath_by_Type,on="type")

#9.04 Calculate the total of students that passed Reading (70 or higher)
PassRead_by_Type=Schools_Students_df.loc[Per_School_Size["reading_score"]>69,:]
PassRead_by_Type=PassRead_by_Type.groupby(["type"],as_index=False).count()
PassRead_by_Type=PassRead_by_Type.loc[:,["type","size"]]
PassRead_by_Type=PassRead_by_Type.rename(columns={"size":"Students Passing Reading"})

#9.04.01 Merges with the main table (Scores_by_Type)
Scores_by_Type=Scores_by_Type.merge(PassRead_by_Type,on="type")

#9.05 Calculates % of Students that passed Math
Scores_by_Type["% Passing Math"]=Scores_by_Type["Students Passing Math"]/Scores_by_Type["size"]

#9.06 Calculates % of Students that passed Reading
Scores_by_Type["% Passing Reading"]=Scores_by_Type["Students Passing Reading"]/Scores_by_Type["size"]

#9.07 Calculate the Overall Passing Rate
Scores_by_Type["Overall Passing Rate"]=(Scores_by_Type["% Passing Math"]+Scores_by_Type["% Passing Reading"])/2

#9.08 Keeps only necessary columns
Scores_by_Type=Scores_by_Type.loc[:,["type","reading_score","math_score","% Passing Math","% Passing Reading",
                                    "Overall Passing Rate"]]

#9.09 Gives Format to the Table

#9.09.1 % Passing Math, % Passing Reading and Overall Passing Rate as Percentage ("{0:.3%}")
Scores_by_Type["% Passing Math"]=Scores_by_Type["% Passing Math"].map("{0:.3%}".format)
Scores_by_Type["% Passing Reading"]=Scores_by_Type["% Passing Reading"].map("{0:.3%}".format)
Scores_by_Type["Overall Passing Rate"]=Scores_by_Type["Overall Passing Rate"].map("{0:.3%}".format)

#9.09.2 Renames Columns
Scores_by_Type=Scores_by_Type.rename(columns={"reading_score":"Avg Reading Score", "math_score":"Avg Math Score"})

#9.09.03 Defines Type as index
Scores_by_Type=Scores_by_Type.set_index("type")

Scores_by_Type
#AVG_by_Type
#PassMath_by_Type
#PassRead_by_Type

,Avg Reading Score,Avg Math Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.902821,83.406183,93.702%,96.646%,95.174%
District,80.962485,76.987026,66.518%,80.905%,73.712%
